In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools

In [ ]:
from czitools.metadata_tools import czi_metadata as czimd
from czitools.read_tools import read_tools
from pylibCZIrw import czi as pyczi
import os
from tqdm.contrib.itertools import product
import numpy as np
from pathlib import Path
import requests

In [ ]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [ ]:
if IN_COLAB:
    filepath = os.path.join(os.getcwd(), "data/CellDivision_T3_Z5_CH2_X240_Y170.czi")
    #zarr_path = Path(filepath[:-4] + ".ome.zarr")

if not IN_COLAB:
    defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
    filepath = os.path.join(defaultdir, "CellDivision_T3_Z5_CH2_X240_Y170.czi")
    #zarr_path = defaultdir / Path(filepath[:-4] + ".ome.zarr")

# return 6D array with dimension order STCZYXA
array6d, mdata  = read_tools.read_6darray(filepath)

## ZSTD compression

* zstd0 - You can specify the parameters of it like "explicitlevel" between -131072 and 22. (e.g. "zstd0:ExplicitLevel=10")
* zstd1 - You can specify the parameters of it like "explicitlevel" between -131072 and 22 or "preprocess" - only have HiLoByteUnpack.
  * This is only done for zstd1 and for pixel types gray16 and Bgr48. (e.g. "zstd1:ExplicitLevel=10" or "zstd1:Preprocess=HiLoByteUnpack")

To read more -: https://facebook.github.io/zstd/

In [ ]:
# define the compression parameters

#cmp = "zstd0:ExplicitLevel=3"
#cmp = "zstd1:ExplicitLevel=10"
#cmp = "zstd1:Preprocess=HiLoByteUnpack;ExplicitLevel=3"
cmp = "zstd1:Preprocess=HiLoByteUnpack;Level=Best"
#cmp = "zstd1:Preprocess=HiLoByteUnpack;ExplicitLevel=1"
#cmp = "uncompressed:"

# create the filename for the new CZI image file
filepath_compressed = os.path.join(os.getcwd(), "newCZI_compressed.czi")
filepath_uncompressed = os.path.join(os.getcwd(), "newCZI_uncompressed.czi")

print(f"Filepath CZI file without compression: {filepath_uncompressed}")
print(f"Filepath CZI file with compression {cmp}: {filepath_compressed}")

remove_czi = True


In [ ]:
# write the same CZI image back to disk but this time with using ZSTD compression

with pyczi.create_czi(filepath_uncompressed, exist_ok=True, compression_options="uncompressed:") as czidoc_w, \
    pyczi.create_czi(filepath_compressed, compression_options=cmp, exist_ok=True) as czidoc_wc:

        # iterate over all 2D planes
        for s, t, c, z in product(range(array6d.shape[0]),
                                range(array6d.shape[1]),
                                range(array6d.shape[2]),
                                range(array6d.shape[3])):

            # writing the image without compression
            czidoc_w.write(data=array6d[s, t, c, z, ...],
                        plane={"C": c, "T": t, "Z": z},
                        scene=s,
                        #compression_options=cmp,  # valid for individual planes
                        )

            # writing the image with compression
            czidoc_wc.write(data=array6d[s, t, c, z, ...],
                        plane={"C": c, "T": t, "Z": z},
                        scene=s)

size = os.path.getsize(filepath_uncompressed)/1024.0**2
size_comp = os.path.getsize(filepath_compressed)/1024.0**2
print(f"Size of the uncompressed CZI file: {np.round(size, 2)} MB ")
print(f"Size of the compressed CZI: {np.round(size_comp, 2)} MB ")
print(f"ZSTD Compression Ratio: {np.round(size/size_comp, 2)}")

In [ ]:
if remove_czi:
    # Clean up: Remove the created CZI files
    from pathlib import Path

    # Convert to Path objects
    path_uncompressed = Path(filepath_uncompressed)
    path_compressed = Path(filepath_compressed)

    if path_uncompressed.exists():
        path_uncompressed.unlink()
        print(f"Removed: {path_uncompressed}")
    else:
        print(f"File not found: {path_uncompressed}")

    if path_compressed.exists():
        path_compressed.unlink()
        print(f"Removed: {path_compressed}")
    else:
        print(f"File not found: {path_compressed}")

    print("Cleanup complete!")
